In [ ]:
%matplotlib inline

# Create result from case study

In [ ]:
from pyglotaran_examples.ex_two_datasets.ex_two_datasets import result

# Data + fit

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
import numpy as np
from glotaran.project.result import Result
from matplotlib.pyplot import Axes

In [ ]:
def plot_data_and_fits(
    result: Result, wavelength: float, axis: Axes, linlog=False, linthresh=1, use_label=True
):
    for dataset_name in result.data.keys():
        result_data = result.data[dataset_name].sel(spectral=[wavelength], method="nearest")
        # result_data.data.plot(
        #    x="time",
        #    ax=axis,
        #    marker="o",
        #    linewidth=0,
        #    label=f"{dataset_name}_data" if use_label else "",
        # )
        result_data.data.plot(x="time", ax=axis, label=f"{dataset_name}_data" if use_label else "")
        result_data.fitted_data.plot(
            x="time", ax=axis, label=f"{dataset_name}_fit" if use_label else ""
        )
    if linlog:
        axis.set_xscale("symlog", linthresh=linthresh)
    axis.set_ylabel("Intensity")

In [ ]:
fig, axes = plt.subplots(2, 1, figsize=(30, 15))
fig.patch.set_facecolor("white")
plot_data_and_fits(result=result, wavelength=400, axis=axes[0], linlog=True)
plot_data_and_fits(result=result, wavelength=501, axis=axes[1], linlog=True)
plt.tight_layout()

In [ ]:
result.data["dataset1"].coords["spectral"]

In [ ]:
result.data["dataset2"].coords["spectral"].values.min()

In [ ]:
def get_overlap(result: Result, coord_name: str = "spectral"):
    minima = []
    maxima = []
    for dataset in result.data.values():
        coord = dataset.coords[coord_name]
        minima.append(coord.values.min())
        maxima.append(coord.values.max())
    return min(minima), max(maxima)


get_overlap(result)

In [ ]:
axes.flatten()

# Fit multiple traces

In [ ]:
def plot_fit_overview(
    result: Result, axes_shape=(4, 4), linlog=False, linthresh=1, figsize=(30, 15), *args, **kwargs
):
    fig, axes = plt.subplots(*axes_shape, figsize=figsize)
    fig.patch.set_facecolor("white")
    wavelengths = np.linspace(*get_overlap(result), num=len(axes.flatten()))
    use_label = True
    for wavelength, axis in zip(wavelengths, axes.flatten()):
        plot_data_and_fits(
            result=result,
            wavelength=wavelength,
            axis=axis,
            linlog=linlog,
            linthresh=linthresh,
            use_label=use_label,
        )
        use_label = False
    fig.legend()
    fig.suptitle("Fit overview", fontsize=28)
    fig.tight_layout()
    return fig, axes


fig, axes = plot_fit_overview(result, axes_shape=(4, 4), linlog=True, linthresh=1)

# POC for plotting plugin system

In [ ]:
plot_overview = plot_fit_overview


class PlotWrapper:
    __default_plot = staticmethod(plot_overview)
    overview = staticmethod(plot_overview)

    def __init__(self, result: Result):
        self.result = result
        result.plot = self

    def __call__(self, *args, **kwargs):
        return self.__default_plot(self.result, *args, **kwargs)

In [ ]:
PlotWrapper(result)

In [ ]:
result.plot();

In [ ]:
# Needs extra wrapping function to result gets automatically injected
result.plot.overview(result);

In [ ]:
help(result.data["dataset1"])